Asked ChatGPT how to convert bigrams into a network and it showed me the workflow which I corrected for errors and adapted to my data

In [1]:
from pyspark.sql import SparkSession
from graphframes import *
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from pyspark.sql import types as T

spark = SparkSession \
        .builder \
        .appName("network") \
        .getOrCreate()

spark_nlp = sparknlp.start()

24/11/19 20:20:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/11/19 20:20:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
data = spark.read.csv("../data_topicmodel.csv", header= True).select(["id", "cleaned_text"])
data = data.sample(0.001)

Preprocess and get tokens

In [3]:
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc", "even", "one", "bye", "take"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year", "now"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot', "selfimprovement", "_"]

stopwords = english + time + reddit + topic_specific

In [4]:
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

In [5]:
tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

In [6]:
normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized')

In [7]:
from sparknlp.base import Finisher

finisher = Finisher().setInputCols(['normalized'])

In [8]:
my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          finisher
      ])

In [9]:
pipelineModel = my_pipeline.fit(data)
processed_data = pipelineModel.transform(data)
processed_data.show(1)

+------+--------------------+--------------------+
|    id|        cleaned_text| finished_normalized|
+------+--------------------+--------------------+
|1nfprr|im feeling pumped...|[im, feeling, pum...|
+------+--------------------+--------------------+
only showing top 1 row



Create bigrams

In [10]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

def generate_bigrams(tokens):
    return [(tokens[i], tokens[i+1]) for i in range(len(tokens) - 1) \
           if tokens[i] not in stopwords and tokens[i+1] not in stopwords]
                    
bigram_udf = F.udf(generate_bigrams, T.ArrayType(T.ArrayType(T.StringType())))
bigrams_df = processed_data.withColumn("bigrams", bigram_udf(F.col("finished_normalized")))

bigrams_df.select('bigrams').show(10, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                             bigrams|
+----------------------------------------------------------------------------------------------------+
|[[feeling, pumped], [walked, miles], [miles, washed], [washed, dishes], [healthy, keto], [keto, b...|
|[[example, learning], [language, musical], [musical, instrument], [instrument, dancing], [dancing...|
|[[feeling, extremely], [sport, lack], [social, contact], [enough, sleep], [sleep, neglected], [ne...|
|[[notes, storyboards], [storyboards, profiles], [vast, contexts], [characters, sketched], [import...|
|[[members, share], [play, guitar], [guys, share], [whatever, resources], [eachother, learn], [sim...|
|[[fall, flat], [flat, every], [top, public], [public, university], [university, business], [busin...|
|[[title, came], [michael, franti], [arts, seem], [understand, human], [h

Create network

In [11]:
edges_df = bigrams_df.select("id", F.explode(F.col("bigrams")).alias("bigram"))
edges_df = edges_df.select(
    F.col("bigram")[0].alias("src"),
    F.col("bigram")[1].alias("dst")
)

In [12]:
edges_df.show(20)

+----------+-----------+
|       src|        dst|
+----------+-----------+
|   feeling|     pumped|
|    walked|      miles|
|     miles|     washed|
|    washed|     dishes|
|   healthy|       keto|
|      keto|  breakfast|
|   studied|    spanish|
|     forty|    minutes|
|    weight|   training|
|   example|   learning|
|  language|    musical|
|   musical| instrument|
|instrument|    dancing|
|   dancing| perfecting|
|      golf|      swing|
|     swing|   learning|
|      edit|     videos|
|    videos|        ect|
|   similar|experiences|
|     learn|     skills|
+----------+-----------+
only showing top 20 rows



Create vertices

In [24]:
vertices_df = edges_df.select(F.col("src").alias("id")).union(edges_df.select(F.col("dst").alias("id"))).distinct()
vertices_df.write.csv("vertices")

In [14]:
vertices_df.show(10)

+---------+
|       id|
+---------+
|      art|
|   doubts|
|    often|
|   taking|
| slightly|
|    staff|
| pleasing|
|  achieve|
|   growth|
|character|
+---------+
only showing top 10 rows



In [25]:
edges_df = edges_df.distinct()
edges_df.write.csv("edges")
edges_df.show(10)

+----------+-------------+
|       src|          dst|
+----------+-------------+
|      adhd|        forum|
|  calculus|   statistics|
|      find|     somebody|
|everything|     thinking|
|   gaining|    admission|
|      kind|   empathetic|
|  slightly|        rural|
|      rude|condescending|
|    public|      stutter|
|      gets|       burned|
+----------+-------------+
only showing top 10 rows



In [23]:
network = GraphFrame(vertices_df, edges_df)
network

GraphFrame(v:[id: string], e:[src: string, dst: string])

In [17]:
network.inDegrees.orderBy(F.col("inDegree"), ascending=False).show()

/software/spark-3.3.2-el8-x86_64/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+-------------+--------+
|           id|inDegree|
+-------------+--------+
|         back|      35|
|        every|      35|
|          try|      26|
|        years|      21|
|          job|      20|
|      instead|      20|
|       enough|      19|
|         away|      19|
|       social|      18|
|       others|      18|
|      working|      17|
|        never|      16|
|       better|      16|
|    sometimes|      15|
|      friends|      15|
|         part|      15|
|       though|      15|
|relationships|      14|
|         find|      14|
|      looking|      14|
+-------------+--------+
only showing top 20 rows



In [22]:
results = network.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").orderBy(F.col("pagerank"), ascending=False).show(50)

+-------------+------------------+
|           id|          pagerank|
+-------------+------------------+
|       afraid| 138.8473000454674|
|       traits|107.15157107215202|
|     earplugs| 68.31217527158195|
|   cancelling|63.056041149254945|
|         wear| 61.97601007507923|
|        noise| 51.51325640431438|
|       metime| 48.16809689614223|
|          try|19.985857062608197|
|         back|17.349171891157592|
|        every|15.696328156309653|
|    sometimes|15.351313008245628|
|      instead|11.186680982653346|
|        makes|10.793549750015732|
|       enough|10.519353161000959|
|         away| 9.988701196503651|
|     whatever| 9.620476982063243|
|      feeling| 9.574432976480624|
|        years|  9.38610903391197|
|      working| 8.614637743122639|
|        first| 8.593862991328285|
|        comes| 8.438233344483548|
|        hours|  8.20238005091758|
|       though|  8.08901210341723|
|        arent| 7.696359095687199|
|        raise| 7.475829912239985|
|          job| 7.43

## REMOVE SAMPLE